In [ ]:
import subprocess

# customMINIAOD generation
CONFIGFILE = 'customMiniAODv2_cfg.py'
NTHREADS   = 4
INPUTFILE  = 'file:' + current_file_path
OUTPUTFILE = INPUTFILE.replace('AODSIM', 'customMINIAODSIM')
COMMAND = 'cmsRun -n ' + str(NTHREADS) + ' ' + CONFIGFILE + ' inputFiles=' + INPUTFILE + ' outputFile=' + OUTPUTFILE + ' maxEvents=-1'
print(COMMAND)
print('-'*80)


logfile = OUTPUTFILE.replace('.root', '.log').replace('file:', '')
f = open(logfile,'w')
loglist.append(f)

subprocess.Popen(COMMAND.split(' '), stderr=f)

In [ ]:
EVCUTS   = {GenJet_pt[0]==200}
SVCUTS   = 
TRCUTS   = 
SSAMPLES = 
BSAMPLES = 
PLOTNAME = 
OUTLOG   = 
NTHREADS = 4


In [13]:
operators = '><='
cutdict_ev = {'RawMET' : 'RawMET_pt'}
mystr = 'RawMET>=2.0'
splitted = None
for operator in operators:
    if operator in mystr:
        splitted = mystr.split(operator, 1)
        splitted[0] = cutdict_ev[splitted[0]]
        splitted[1] = operator + splitted[-1]
        break
print(''.join(splitted))

RawMET_pt>=2.0


In [20]:
from itertools import product

def grid_parameters(parameters):
    for params in product(*parameters.values()):
        yield dict(zip(parameters.keys(), params))

In [46]:
cutdict_ev['evRawMET'] = ('RawMET_pt',     '>', 200.)
cutdict_ev['evGenMET'] = ('GenMET_pt',     '>', 200.)
cutdict_ev['evGenJet'] = ('GenJet_pt[0]]', '>', 100.)

cutdict_sv['svDPHIRM'] = ('acos(cos(SDVSecVtx_L_phi-GenMET_phi))', '<', 1.5)
cutdict_sv['svRALPHA'] = ('SDVSecVtx_pAngle', '>', 0.2)
cutdict_sv['svLXYSIG'] = ('SDVSecVtx_LxySig', '>', 20)
cutdict_sv['svNGTR'] =   ('SDVSecVtx_newgoodtr', '>=', 2)
cutdict_sv["svDPHIJ"] =  ('acos(cos(SDVSecVtx_L_phi-GenJet_phi[0]))]', '>', 1)
cutdict_sv["svNDOF"] =   ('SDVSecVtx_ndof>1', '', )

cutdict_tr['trDXYSIG'] = ('(abs(SDVTrack_dxy)/SDVTrack_dxyError)>4', '', )
cutdict_tr['trCHI2'] =   ('SDVTrack_normalizedChi2 < 5', '', )
cutdict_tr['trDZ'] =     ('abs(SDVTrack_dz)<4.', '', )
cutdict_tr['trNVH'] =    ('SDVTrack_numberOfValidHits>13', '', )
cutdict_tr['trRSPT'] =   ('(SDVTrack_ptError/SDVTrack_pt)<0.015', '', )
cutdict_tr["trDPHIJ"] =  ('acos(cos(SDVTrack_phi-GenJet_phi[0]))>1', '', )
cutdict_tr["trDPHIM"] =  ('acos(cos(SDVTrack_phi-GenMET_phi))<1.5', '', )









grid_cuts = dict()
grid_cuts['RawMET>'] = (100, 200, 400, 800)
grid_cuts['SDVSecVtx_LxySig>'] = (10, 12, 14, 18)
grid_cuts['SDVTrack_dz<'] = (1, 2, 4, 8)

for grid in grid_parameters(grid_cuts):
    # print(grid.items())
    grid_as_str = (''.join((key,str(value))) for key, value in grid.items())
    print('&&'.join(grid_as_str))

RawMET>100&&SDVTrack_dxy>1&&SDVTrack_dz<1
RawMET>100&&SDVTrack_dxy>1&&SDVTrack_dz<2
RawMET>100&&SDVTrack_dxy>1&&SDVTrack_dz<4
RawMET>100&&SDVTrack_dxy>1&&SDVTrack_dz<8
RawMET>100&&SDVTrack_dxy>2&&SDVTrack_dz<1
RawMET>100&&SDVTrack_dxy>2&&SDVTrack_dz<2
RawMET>100&&SDVTrack_dxy>2&&SDVTrack_dz<4
RawMET>100&&SDVTrack_dxy>2&&SDVTrack_dz<8
RawMET>100&&SDVTrack_dxy>4&&SDVTrack_dz<1
RawMET>100&&SDVTrack_dxy>4&&SDVTrack_dz<2
RawMET>100&&SDVTrack_dxy>4&&SDVTrack_dz<4
RawMET>100&&SDVTrack_dxy>4&&SDVTrack_dz<8
RawMET>100&&SDVTrack_dxy>8&&SDVTrack_dz<1
RawMET>100&&SDVTrack_dxy>8&&SDVTrack_dz<2
RawMET>100&&SDVTrack_dxy>8&&SDVTrack_dz<4
RawMET>100&&SDVTrack_dxy>8&&SDVTrack_dz<8
RawMET>200&&SDVTrack_dxy>1&&SDVTrack_dz<1
RawMET>200&&SDVTrack_dxy>1&&SDVTrack_dz<2
RawMET>200&&SDVTrack_dxy>1&&SDVTrack_dz<4
RawMET>200&&SDVTrack_dxy>1&&SDVTrack_dz<8
RawMET>200&&SDVTrack_dxy>2&&SDVTrack_dz<1
RawMET>200&&SDVTrack_dxy>2&&SDVTrack_dz<2
RawMET>200&&SDVTrack_dxy>2&&SDVTrack_dz<4
RawMET>200&&SDVTrack_dxy>2&&SDVTra